In [ ]:
import numpy as np
!pip install opencv-python
import cv2


def colorPred(num):
    assert num in range(0,11)

    # buildings & manmade structures
    if num < 2: return (255,255,255)
    # Road & Track
    if num < 4: return (255,255,0)
    # Trees & Crops
    if num < 6: return (0,255,0)
    # Waterway & Standing water
    if num < 8: return (0,0, 255)
    # Vehicle Large & Small
    if num < 10: return (0,255,255)
    # unspecified
    if num == 10: return (0,0,0)

#split data to batches for SGD
def batchSplitter(batchSize, x, y=None):
    nBatch = int(np.ceil(len(x) / float(batchSize)))
    #create a random sequence of indices to 'shuffle' x
    permutation = np.random.permutation(len(x))
    
    if y is None:
        xShuffled = x[permutation]
        for i in range(nBatch):
            first = i * batchSize
            last = first + batchSize
            yield xShuffled[first:last]
    else:
        xShuffled = x[permutation]
        yShuffled = y[permutation]
        for i in range(nBatch):
            first = i * batchSize
            last = first + batchSize
            yield xShuffled[first:last], yShuffled[first:last]


def inverseFlatten(pred):
    import matplotlib.pyplot as plt
    assert type(pred) == list
    sz = int(np.sqrt(len(pred)))
    assert sz == 478
    img = np.zeros((sz,sz,3))
    # f = lambda arr: [colorPred(arr[i]) for i in range(len(arr))]
    # img = [f(pred[i*sz : (i+1)*sz]) for i in range(sz)]
    for i in range(sz):
        for j in range(sz):
            img[i][j] = colorPred(pred[i*sz+j])
    img = cv2.resize(img, (800, 800))
    return img
def showOriginal(id):
    import matplotlib.pyplot as plt
    def getImage(image_id, dims=3, size=800, case=""):
        filename = "./testImages/{}.tif".format(image_id)
        img = tiff.imread(filename)
        img = np.rollaxis(img, 0, 3)
        img = cv2.resize(img, (size, size))
        return img
    def ContrastAdjustment(img, low_p = 5, high_p = 95):
        img_adjusted = np.zeros_like(img, dtype=np.float32)
        for i in range(img.shape[2]):
            mn, mx = 0, 1       # np.min(img), np.max(img)
            p_low, p_high = np.percentile(img[:, :, i], low_p), np.percentile(img[:, :, i], high_p)
            tmp = mn + (img[:, :, i] - p_low) * (mx - mn) / (p_high - p_low)
            tmp[tmp < mn] = mn
            tmp[tmp > mx] = mx
            img_adjusted[:, :, i] = tmp
        return img_adjusted.astype(np.float32)

    img = getImage(id)
    img = ContrastAdjustment(img)
    plt.figure()
    #ax1 = plt.subplot(131)
    plt.title('image ID: {}'.format(id))
    plt.imshow(img[:, :, :3])
    plt.show()


In [ ]:
!pip install tifffile
import tifffile as tiff
import cv2
import numpy as np

def computeNormal2D(mat): #input must be a 2D array
    amin = np.amin(mat)
    mat = (mat - amin) / (np.amax(mat) - amin)
    return mat

def computeNormal3D(mat): #input must be a 3D array
    for i in range(mat.shape[2]):
        mat[:,:,i] = computeNormal2D(mat[:,:,i])
    return mat

def computeSMoment(mat,mean=None): #input must be a 2D array
    if mean == None:
        mean = mat.mean()
    return np.sqrt(mean)/np.square(mat).mean()

def convertRGB2HSV(img):
    img = img/1023.0
    imgHSV = img
    for i in range(len(img)):
        for j in range(len(img[0])):
            cmax = max(img[i][j][0],img[i][j][1],img[i][j][2])
            cmin = min(img[i][j][0],img[i][j][1],img[i][j][2])
            delta = cmax-cmin

            if cmax==cmin:
                imgHSV[i][j][0] = 0
            elif cmax == img[i][j][0]:
                imgHSV[i][j][0] = (60 * ((img[i][j][1] - img[i][j][2]) / delta) + 360) % 360
            elif cmax == img[i][j][1]:
                imgHSV[i][j][0] = (60 * ((img[i][j][2] - img[i][j][0]) / delta) + 120) % 360
            elif cmax == img[i][j][2]:
                imgHSV[i][j][0] = (60 * ((img[i][j][0] - img[i][j][1]) / delta) + 240) % 360
      
            if cmax==0:
                imgHSV[i][j][1]=0
            else:
                imgHSV[i][j][1]=(delta/cmax)
      
            imgHSV[i][j][2]=cmax

    return imgHSV

def computeCCMFeatures(img,band,windowSize = 7):

    if windowSize % 2 == 0:
        return
    paramCount = 0
    if band == 'H':
        paramCount = 3
        img = img*255/360
    elif band == 'S':
        paramCount = 2
        img = img*255
    elif band == 'V':
        paramCount = 2
        img = img*255
    else:
        return
  
    img = np.around(img)  
    img = img.astype(np.uint8)
  
    newImg = np.zeros((478,478,paramCount))
    halfWindow=windowSize//2
    ccm = np.zeros((256,256), dtype=np.uint8) #assuming windowSize <= 15
    a = 0
    b = 0
    for i in range(halfWindow,img.shape[0]-halfWindow,windowSize):
        for j in range(halfWindow,img.shape[1]-halfWindow,windowSize):
            # i,j = pixel in whole-image loop
            ccm = np.zeros((256,256), dtype=np.uint8) #assuming windowSize <= 15
            for k in range(i-halfWindow,min(img.shape[0]-1,i+halfWindow+1)):
                for l in range(j-halfWindow,min(img.shape[1]-1,j+halfWindow+1)):
                    # k,l = pixel in window loop
                    ccm[img[k,l],img[k+1,l+1]] += 1
            mean = ccm.mean()
            newImg[a][b][0] = mean
            ccmSquared = np.square(ccm)
        
            if band == 'H':
                #sosvh is SSD
                newImg[a][b][1]=np.sum(np.square(ccm-mean)) #sosvh
                newImg[a][b][2]=np.sum(ccmSquared) #autoc
            else:
                newImg[a][b][1]=(np.sqrt(mean))/(ccmSquared.mean()) #smoment
                #if band == 'V':
                #  covariance = np.cov(ccm)[0][-1]
                #  newImg[a][b][2]=covariance
            b = (b+1)%478
        a += 1
    return newImg


def computeHSVFeatures(img,windowSize = 7):
    paramCount = 7
    newImg = np.zeros((478,478,paramCount))
    halfWindow=windowSize//2
    a = 0
    b = 0
    for i in range(halfWindow,img.shape[0]-halfWindow,7):
        for j in range(halfWindow,img.shape[1]-halfWindow,7):
            # i,j = pixel in whole-image loop      
        
            minX=i-halfWindow
            maxX=i+halfWindow+1
            minY=j-halfWindow
            maxY=j+halfWindow+1

            newImg[a][b][4] = img[minX:maxX,minY:maxY,0].mean() #meanH
            newImg[a][b][6] = img[minX:maxX,minY:maxY,1].mean() #meanS
            newImg[a][b][5] = img[minX:maxX,minY:maxY,2].mean() #meanV
            newImg[a][b][0] = computeSMoment(img[minX:maxX,minY:maxY,2], newImg[a][b][5]) #smomentV
            newImg[a][b][1] = np.var(img[minX:maxX,minY:maxY,2]) #varianceV
            newImg[a][b][2] = np.sqrt(newImg[a][b][1]) #stdV
            newImg[a][b][3] = np.std(img[minX:maxX,minY:maxY,0]) #stdH
            b = (b+1)%478
        a += 1
        
    return newImg

def computeNIRFeatures(img,windowSize = 7):
    paramCount = 2
    newImg = np.zeros((478,478,paramCount))
    halfWindow=windowSize//2
    a = 0
    b = 0
    for i in range(halfWindow,img.shape[0]-halfWindow,7):
        for j in range(halfWindow,img.shape[1]-halfWindow,7):
            # i,j = pixel in whole-image loop      
            
            minX=i-halfWindow
            maxX=i+halfWindow+1
            minY=j-halfWindow
            maxY=j+halfWindow+1
            
            newImg[a][b][1] = img[minX:maxX,minY:maxY].mean() #meanNIR
            newImg[a][b][0] = np.sqrt(np.var(img[minX:maxX,minY:maxY])) #stdNIR
            b = (b+1)%478
        a += 1
    
    return newImg

def extractImageFeatures(imagePath):
    imgRGBN = np.zeros((3346, 3346, 4), "float32")

    imgRGBN[..., 3] = cv2.resize(np.transpose(tiff.imread("{}_M.tif".format(imagePath[:-4])), (1,2,0))[:,:,7], (3346, 3346))

    imgRGBN[..., 0:3] = cv2.resize(np.moveaxis(tiff.imread(imagePath),0,-1), (3346, 3346)) #compress to fit 7*7

    imgHSV = convertRGB2HSV(imgRGBN[:,:,0:3])
    img = np.zeros((478,478,16)) 
    
    temp = computeCCMFeatures(imgHSV[:,:,0],'H')
    #print("ccmH")
    img[:,:,4]=temp[:,:,0]
    img[:,:,1]=temp[:,:,1]
    img[:,:,2]=temp[:,:,2]

    temp = computeCCMFeatures(imgHSV[:,:,1],'S')
    #print("ccmS")
    img[:,:,3]=temp[:,:,0]
    img[:,:,5]=temp[:,:,1]

    temp = computeCCMFeatures(imgHSV[:,:,2],'V')
    #print("ccmI")
    img[:,:,0]=temp[:,:,0]
    img[:,:,6]=temp[:,:,1]

    temp = computeHSVFeatures(imgHSV)
    #print("HSV")
    img[:,:,7:9]=temp[:,:,0:2]
    img[:,:,10:15]=temp[:,:,2:7]

    temp = computeNIRFeatures(imgRGBN[...,3])
    #print("NIR")
    img[:,:,9]=temp[:,:,0]
    img[:,:,15]=temp[:,:,1]

    return img

def flattenImage(mat): #input must be a 3D array
    flatMat = np.zeros((mat.shape[0]*mat.shape[1],mat.shape[2]))
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            flatMat[i*mat.shape[0]+j]=mat[i,j]
    return flatMat

def readImageDBN(imagePath):
    img = extractImageFeatures(imagePath)
    img = computeNormal3D(img)
    img = flattenImage(img)
    return img

In [ ]:
chosen_path = "/content/drive/My Drive/GP/dstl/walid/"

def concatFeatureVectors():
    s = 478 #478 is my image size
    x = np.zeros((5 * s, 5 * s, 16))
    #x = np.load((chosen_path+'x_trn_10.npy'))

    ids = sorted(tr_wkt.ImageId.unique())
    print(len(ids))
    for i in range(5):
        for j in range(5):
            if np.amax(x[s * i:s * i + s, s * j:s * j + s, :]) == 0:
                id = ids[5 * i + j]
                img = extractImageFeatures(id)
                
                x[s * i:s * i + s, s * j:s * j + s, :] = img[:s, :s, :]
                np.save((chosen_path+'x_trn_10'), x)
            print(5 * i + j)
            
    np.save((chosen_path+'x_trn_10'), x)

In [ ]:
import numpy as np
!pip install tensorflow==2.0.0
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.disable_v2_behavior()

sess = tf.Session()

class RBM():
    def __init__(self,
                 nHidden=200,
                 learningRate=1e-3,
                 nEpochs=5,
                 nIterCD=1,
                 batchSize=256,
                 verbose=True):
        self.nHidden = nHidden
        self.learningRate = learningRate
        self.nEpochs = nEpochs
        self.nIterCD = nIterCD
        self.batchSize = batchSize
        self.verbose = verbose

    @classmethod
    def variableNames(cls):
        return ['nHidden',
                'nVisible',
                'learningRate',
                'nEpochs',
                'nIterCD',
                'batchSize',
                'verbose']

    # Inititializes an object with values loaded from a file
    @classmethod
    def initFromDict(cls, externalDict):
        weights = {name: externalDict.pop(name) for name in ['W', 'b', 'a']}

        nVisible = externalDict.pop('nVisible')
        newObject = cls(**externalDict)
        setattr(newObject, 'nVisible', nVisible)

        # Initialize RBM
        newObject.defineModel(weights)
        sess.run(tf.variables_initializer([getattr(newObject, name) for name in ['W', 'b', 'a']]))

        return newObject

    # Gets the object's variables to save them in a file
    def saveToDict(self):
        internalDict = {name: self.__getattribute__(name) for name in self.variableNames()}
        internalDict.update({name: self.__getattribute__(name).eval(sess) for name in ['W', 'b', 'a']})
        return internalDict

    def defineModel(self, weights=None):
        # Initialize the weights and biases
        if weights:
            for attr_name, value in weights.items():
                self.__setattr__(attr_name, tf.Variable(value))
        else:            
            std = 1.0 / np.sqrt(self.nVisible)
            self.W = tf.Variable(tf.random_normal([self.nHidden, self.nVisible], std))
            self.b = tf.Variable(tf.random_normal([self.nHidden], std))
            self.a = tf.Variable(tf.random_normal([self.nVisible], std))

        # TensorFlow operations
        # Assign the visible nodes to a placeholder
        self.nodesVisible = tf.placeholder(tf.float32, shape=[None, self.nVisible])
        # Forward Pass - Calculate the hidden nodes: H0 = sigmoid(((W x V^T) + c|b)^T)
        self.calcH0 = tf.nn.sigmoid(tf.transpose(tf.matmul(self.W, tf.transpose(self.nodesVisible))) + self.b)
        # Assign the hidden nodes to a placeholder 
        self.nodesHidden = tf.placeholder(tf.float32, shape=[None, self.nHidden])
        # Backward Pass - Approximate the visible nodes: V' = sigmoid((H x W)+ b|a)
        self.calcV0 = tf.nn.sigmoid(tf.matmul(self.nodesHidden, self.W) + self.a)
        # Create a matrix RUV0 like H with normaly distributed random variables
        RUV = tf.Variable(tf.random_uniform([self.batchSize, self.nHidden]))
        # Choose sample values Hs0 from H0 using the RUV0
        sampleH0 = tf.cast(RUV < self.calcH0, 'float32')
        # append the RUV0 to a list of all RUVs
        self.randomVariables = [RUV]

        # we multiply the sample hidden values Hs0 by the visible values V
        # [B, H, 1] x [B, 1, V] = [B, H, V]
        positiveGradient = tf.matmul(tf.expand_dims(sampleH0, 2), tf.expand_dims(self.nodesVisible, 1))

        # Negative gradient
        # Gibbs sampling
        sampleHi = sampleH0
        for i in range(self.nIterCD):
            # V1 = sig Hs0 x W + a
            calcVi = tf.nn.sigmoid(tf.matmul(sampleHi, self.W) + self.a)
            # Hi = sigmoid(((W x V1^T) + b)^T)
            calcHi = tf.nn.sigmoid(tf.transpose(tf.matmul(self.W, tf.transpose(calcVi))) + self.b)
            # create RUVi
            RUV = tf.Variable(tf.random_uniform([self.batchSize, self.nHidden]))
            # Choose sample values Hsi from H1 using the RUVi
            sampleHi = tf.cast(RUV < calcHi, 'float32')
            # append the RUVi
            self.randomVariables.append(RUV)
        
        # [B, H, 1] x [B, 1, V] = [B, H, V]
        negativeGradient = tf.matmul(tf.expand_dims(sampleHi, 2), tf.expand_dims(calcVi, 1))

        # dW = batchAverage(Hs0 x V0 - Hs1 x V1)
        calcDeltaW = tf.reduce_mean(positiveGradient - negativeGradient, 0)
        # da = batchAverage(V-V1)
        calcDeltaA = tf.reduce_mean(self.nodesVisible - calcVi, 0)
        # db = batchAverage(Hs0-Hs1)
        calcDeltaB = tf.reduce_mean(sampleH0 - sampleHi, 0)

        # W = W + r*dW
        self.iterateW = tf.assign_add(self.W, self.learningRate * calcDeltaW)
        # a = a + r*da
        self.iterateA = tf.assign_add(self.a, self.learningRate * calcDeltaA)
        # b = b + r*db
        self.iterateB = tf.assign_add(self.b, self.learningRate * calcDeltaB)


    def fit(self, X):
        self.nVisible = X.shape[1]
        self.defineModel()
        sess.run(tf.variables_initializer([self.W, self.b, self.a]))  

        for i in range(self.nEpochs):
            idx = np.random.permutation(len(X))
            data = X[idx]
            for batch in batchSplitter(self.batchSize, data):
                if len(batch) < self.batchSize:
                    # Zero Padding
                    pad = np.zeros((self.batchSize - batch.shape[0], batch.shape[1]), dtype=batch.dtype)
                    batch = np.vstack((batch, pad))
                # Get new random variables
                sess.run(tf.variables_initializer(self.randomVariables))
                sess.run([self.iterateW, self.iterateA, self.iterateB],
                         feed_dict={self.nodesVisible: batch})
            if self.verbose:
                print("RBM Epoch", i, "finished.")
        return
    
    # make a forward pass
    def forwardPass(self, X):
        return sess.run(self.calcH0, feed_dict={self.nodesVisible: X})



In [ ]:
class DBN():
    def __init__(self,
                 architectureRBM,
                 learningRateNN,
                 learningRateRBM,
                 nEpochsNN,
                 nEpochsRBM,
                 nIterCD,
                 batchSizeRBM,
                 batchSizeNN,
                 dropout,
                 verbose=True):
        self.architectureRBM = architectureRBM
        self.learningRateNN = learningRateNN
        self.learningRateRBM = learningRateRBM
        self.nEpochsNN = nEpochsNN
        self.nEpochsRBM = nEpochsRBM
        self.nIterCD = nIterCD
        self.batchSizeRBM = batchSizeRBM
        self.batchSizeNN = batchSizeNN
        self.dropout = dropout
        self.verbose = verbose
        self.stackedRBMs = None

    @classmethod
    def variableNames(cls):
        return ['architectureRBM',
                'learningRateRBM',
                'nEpochsRBM',
                'nIterCD',
                'batchSizeRBM',
                'nEpochsNN',
                'learningRateNN',
                'batchSizeNN',
                'dropout',
                'verbose',
                'mapLabel2Index', 
                'mapIndex2Label']

    def save(self, save_path):
        import pickle
        with open(save_path, 'wb') as filePath:
            internalDict = {name: self.__getattribute__(name) for name in self.variableNames()}
            internalDict.update({name: self.__getattribute__(name).eval(sess) for name in ['W', 'b']})
            internalDict['stackedRBMs'] = [rbm.saveToDict() for rbm in self.stackedRBMs]
            internalDict['nClasses'] = self.nClasses
            pickle.dump(internalDict, filePath)

    @classmethod
    def load(cls, load_path):
        import pickle
        with open(load_path, 'rb') as filePath:
            externalDict = pickle.load(filePath)
            weights = {var_name: externalDict.pop(var_name) for var_name in ['W', 'b']}
            nClasses = externalDict.pop('nClasses')
            mapLabel2Index = externalDict.pop('mapLabel2Index')
            mapIndex2Label = externalDict.pop('mapIndex2Label')
            stackedRBMs = externalDict.pop('stackedRBMs')
            
            newObject = cls(**externalDict)
            
            setattr(newObject, 'stackedRBMs', [RBM.initFromDict(rbm) for rbm in stackedRBMs])
            setattr(newObject, 'nClasses', nClasses)
            setattr(newObject, 'mapLabel2Index', mapLabel2Index)
            setattr(newObject, 'mapIndex2Label', mapIndex2Label)
            # Initialize RBM parameters
            newObject.defineModel(weights)
            sess.run(tf.variables_initializer([getattr(newObject, name) for name in ['W', 'b']]))
            return newObject

    # convert class label to mask vector.
    def mapLabels(self, labels, nClasses):
        newLabels = np.zeros([len(labels), nClasses])
        mapLabel2Index, mapIndex2Label = dict(), dict()
        index = 0
        for i, label in enumerate(labels):
            if label not in mapLabel2Index:
                mapIndex2Label[index] = label
                mapLabel2Index[label] = index
                index += 1
            newLabels[i][mapLabel2Index[label]] = 1
        return newLabels, mapLabel2Index, mapIndex2Label

    def defineModel(self, weights=None):
        self.nodesVisible = self.stackedRBMs[0].nodesVisible
        keepProb = tf.placeholder(tf.float32)
        # Apply dropout on the visible nodes
        nodesVisible_drop = tf.nn.dropout(self.nodesVisible, keepProb)
        self.keepProbs = [keepProb]

        # Define tensorflow operation for a forward pass
        self.outputRBM = nodesVisible_drop
        for rbm in self.stackedRBMs:
            self.outputRBM = tf.nn.sigmoid(tf.transpose(tf.matmul(rbm.W, tf.transpose(self.outputRBM))) + rbm.b)
            keepProb = tf.placeholder(tf.float32)
            self.keepProbs.append(keepProb)
            self.outputRBM = tf.nn.dropout(self.outputRBM, keepProb)

        # should be n_nInputNN
        self.nInputNN = self.stackedRBMs[-1].nHidden

        # Initialize the weights and biases
        if weights:
            for attr_name, value in weights.items():
                self.__setattr__(attr_name, tf.Variable(value))
        else:
            std = 1.0 / np.sqrt(self.nInputNN)
            self.W = tf.Variable(tf.random_normal([self.nInputNN, self.nClasses], std))
            self.b = tf.Variable(tf.random_normal([self.nClasses], std))
        
        # Use Stochastic Gradient Descent optimizer and assign the learning rate
        self.optimizerSGD = tf.train.GradientDescentOptimizer(self.learningRateNN)

        # operations
        self.trueY = tf.placeholder(tf.float32, shape=[None, self.nClasses])
        self.predictedY = tf.matmul(self.outputRBM, self.W) + self.b
        self.outputNN = tf.nn.softmax(self.predictedY)
        self.lossFunction = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(tf.stop_gradient(self.trueY), self.predictedY))
        self.trainingStep = self.optimizerSGD.minimize(self.lossFunction)

    def fit(self, X, Y=None):
        #self.pre_train(X)
        self.stackedRBMs = list()
        for nHidden in self.architectureRBM:
            rbm = RBM(nHidden=nHidden,
                            learningRate=self.learningRateRBM,
                            nEpochs=self.nEpochsRBM,
                            nIterCD=self.nIterCD,
                            batchSize=self.batchSizeRBM,
                            verbose=self.verbose)
            self.stackedRBMs.append(rbm)

        # Fit RBM
        if self.verbose:
            print("Unsupervised Learning Phase:")
        inputDataNN = X
        for rbm in self.stackedRBMs:
            rbm.fit(inputDataNN)
            inputDataNN = rbm.forwardPass(inputDataNN)

        # Assign the number of nodes(classes)
        self.nClasses = len(np.unique(Y))
        if self.nClasses == 1:
            Y = np.expand_dims(Y, -1)

        # Build the neural network
        self.defineModel()
        sess.run(tf.variables_initializer([self.W, self.b]))

        # Change given labels to classifier format
        Y, mapLabel2Index, mapIndex2Label = self.mapLabels(Y, self.nClasses)
        self.mapLabel2Index = mapLabel2Index
        self.mapIndex2Label = mapIndex2Label

        if self.verbose:
            print("Supervised Learning Phase:")
        for epoch in range(self.nEpochsNN):
            for batchData, batchLabels in batchSplitter(self.batchSizeNN, X, Y):
                feed_dict = {self.nodesVisible: batchData, self.trueY: batchLabels}
                feed_dict.update({placeholder: (1 - self.dropout) for placeholder in self.keepProbs})
                sess.run(self.trainingStep, feed_dict=feed_dict)

            if self.verbose:
                print("NN Epoch %d finished." % (epoch))
        if self.verbose:
            print("Model finished.")
        return self

    def predict(self, X):
        # Predict probability of each classes for every datapoint
        feed_dict = {self.nodesVisible: X}
        feed_dict.update({placeholder: 1.0 for placeholder in self.keepProbs})
        probs = sess.run(self.outputNN, feed_dict=feed_dict)
        indexes = np.argmax(probs, axis=1)
        # Change network output to given labels
        labels = map(lambda idx: self.mapIndex2Label[idx], indexes)
        return list(labels)

In [ ]:
'''
X = np.load(('/content/drive/My Drive/GP/dstl/walid/x_whole_10_478_16.npy'))
Xnew = np.load(('/content/drive/My Drive/GP/dstl/walid/x_whole_10_478_last.npy'))
X[...,1]=Xnew
#Y = np.load(('/content/drive/My Drive/GP/dstl/walid/y_trn_10.npy'))
X = computeNormal3D(X)
Y2 = np.zeros((Y.shape[0],Y.shape[1]))
for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
        curr = 10
        for k in range(Y.shape[2]):
            if Y[i,j,k] != 0:
                curr = k
        Y2[i,j]=curr
Y2 = Y2.flatten()''';

In [ ]:
#print(X2.shape)
#np.save('/content/drive/My Drive/GP/dstl/walid/x_whole_10_478_16_flat',X2)
#np.save('/content/drive/My Drive/GP/dstl/walid/CDBN_512/y2_trn_10',Y2)
#np.save('/content/drive/My Drive/GP/dstl/walid/x_whole_10_478_16',X)

In [ ]:
#building a model using the labeled dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
X = np.load(('/content/drive/My Drive/GP/dstl/walid/x_whole_10_478_16_flat.npy'))
Y = np.load(('/content/drive/My Drive/GP/dstl/walid/y_whole_10_478_flat.npy'))
X_trn, X_val, Y_trn, Y_val = train_test_split(X, Y, test_size=0.75, random_state=0)
# Training
classifier = DBN(architectureRBM=[100, 100, 100],
                                         learningRateRBM=0.05,
                                         learningRateNN=0.1,
                                         nEpochsRBM=3,
                                         nEpochsNN=100,
                                         nIterCD = 1,
                                         batchSizeRBM=512,
                                         batchSizeNN=64,
                                         dropout=0.2)
classifier.fit(X_trn, Y_trn)
classifier.save('/content/drive/My Drive/GP/dstl/walid/model_512_64_3_25.pk1')
# Test
Y_pred = classifier.predict(X_val)
print('Done. Accuracy with 10 classes: %f' % accuracy_score(Y_val, Y_prd))
Y_test2 = np.zeros_like(Y_val)
Y_pred2 = np.zeros_like(Y_prd)
for i in range(len(Y_test)):
    Y_val2[i] = Y_val[i]//2
    Y_prd2[i] = Y_prd[i]//2
print('Done. Accuracy with 5 classes: %f' % accuracy_score(Y_val2, Y_prd2))

In [ ]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import jaccard_similarity_score    # actually the same as accuracy, doesn't need 'average' argument

classifier = DBN.load('/content/drive/My Drive/GP/dstl/walid/model_512_64_3_25.pk1')
# Splitting data
X = np.load(('/content/drive/My Drive/GP/dstl/walid/x_whole_10_478_16_flat.npy'))
Y = np.load(('/content/drive/My Drive/GP/dstl/walid/y_whole_10_478_flat.npy'))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.75, random_state=0)
Y_pred = classifier.predict(X_test)
print('Done. Jaccard Score with 10 classes: %f' % jaccard_score(Y_test, Y_pred, average="weighted"))
Y_test2 = np.zeros_like(Y_test)
Y_pred2 = np.zeros_like(Y_pred)
for i in range(len(Y_test)):
    Y_test2[i] = Y_test[i]//2
    Y_pred2[i] = Y_pred[i]//2
print('Done. Jaccard Score with 5 classes: %f' % jaccard_score(Y_test2, Y_pred2, average="weighted"))

In [ ]:
img = './testImages/6160_0_0.tif'
classifier = DBN.load('./savedModels/model_512_64_3_25.pk1')
#X = readImageDBN(img)
X = np.load('./savedFeatures/6160_0_0_FE.npy')
Y = classifier.predict(X)
colored_mask = inverseFlatten(Y)

In [ ]:
showOriginal('6160_0_0')

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(colored_mask)
plt.show()